In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local[*]').appName('test').getOrCreate()
sc = spark.sparkContext

In [ ]:
rdd = sc.wholeTextFiles('./tiny_csv')
rdd.map(lambda x:x[1]).collect()

[('file:/Users/wangbo/Desktop/dev/notes/Spark学习/tiny_csv/4.csv',
  'feature1,feature2,feature3\r\n1,4,7\r\n2,5,8\r\n3,6,9\r\n'),
 ('file:/Users/wangbo/Desktop/dev/notes/Spark学习/tiny_csv/1.csv',
  'feature1,feature2,feature3\r\n1,4,7\r\n2,5,8\r\n3,6,9\r\n'),
 ('file:/Users/wangbo/Desktop/dev/notes/Spark学习/tiny_csv/3.csv',
  'feature1,feature2,feature3\r\n1,4,7\r\n2,5,8\r\n3,6,9\r\n'),
 ('file:/Users/wangbo/Desktop/dev/notes/Spark学习/tiny_csv/2.csv',
  'feature1,feature2,feature3\r\n1,4,7\r\n2,5,8\r\n3,6,9\r\n')]